Import Necessary Libraries

In [904]:
import statsapi as api
import pandas as pd
import tempfile
import os
import re
import requests
from bs4 import BeautifulSoup, SoupStrainer
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from datetime import date as d
from datetime import timedelta

Input Player Name and Date


NEED TO CHECK AND FIX ALL CELLS: IF STATS FOR SOMETHING DO NOT EXIST NEED TO DEAL WITH THAT!

ALSO NEED TO ENSURE CORRECT VALUES ARE PASSED IN FOR COMPUTING CORRECT VALUES

NEED TO CHECK PLAYER AVG OVER X GAMES - NEED TO COMPUTE PLAYER AVG OVER LAST GAME FOR PAST SEASONS AS WELL

NEED TO ENSURE THAT ALL DATA IS OF CORRECT TYPE, CORRECT VALUES, CORRECT YEARS, AND IF NONE NEEDS TO BE HANDLED

FIX CURRENT SET UP FOR COMPUTING AVERAGES. SHIT IS BUGGY AS FUCK

IF IT APPEARS TO BE GOOD: THERE WILL BE :P NEXT TO IT. IF NEEDS ADJUSTMENTS, VERIFICATION, OR REWORK, :( NEXT TO IT

ESPECIALLY DECIDE HOW TO DEAL WITH EDGE CASES AND DIVISION BY 0.

WALKS, HIT BY PITCH, AND SAC FLIES DO NOT COUNT AS AN AT BAT

TOTAL BASES ARE ONLY BASES REACHED VIA HITS.

In [905]:
first_name = input("Enter Player First Name: ")
last_name = input('Enter Player Last Name: ')
#date = input('Enter Date: ')
date = '2024-06-04'
year = int(date[:4])
last_year = year - 1
print(year)
player_name = f'{first_name.lower()} {last_name.lower()}'


2024


fetch player_id :P

In [906]:
player_info = api.lookup_player(player_name, season=year)
player_id = player_info[0]['id']
team_id = player_info[0]['currentTeam']['id']
print(player_id)
print(team_id)

660271
119


Gather batter_hand, batter_age, , batter_strike_zone_top, batter_strike_zone_bottom :P

In [907]:
person_info = api.get('people',{'personIds':player_id, 'season':year,'sportId':1})
batter_hand = person_info['people'][0]['batSide']['code']
batter_age = person_info['people'][0]['currentAge']
batter_strike_zone_top = person_info['people'][0]['strikeZoneTop']
batter_strike_zone_bottom = person_info['people'][0]['strikeZoneBottom']
batter_starting_year = int(person_info['people'][0]['mlbDebutDate'][:4])
batter_year = year - batter_starting_year


print(batter_age)
print(batter_year)
print(batter_hand)
print(batter_strike_zone_top)
print(batter_strike_zone_bottom)

29
6
L
3.5
1.59


fetch game_pk, home, opp_id, venue_id, sPitcher_id, sPitcher_hand :P

In [908]:
game_info = api.schedule(date=date,team=team_id,sportId=1)
game_pk = game_info[0]['game_id']
home_team_id = game_info[0]['home_id']
if home_team_id == team_id:
    home = True
    opp_id = game_info[0]['away_id']
    sPitcher_name = game_info[0]['away_probable_pitcher']
else:
    home = False
    opp_id = game_info[0]['home_id']
    sPitcher_name = game_info[0]['home_probable_pitcher']

sPitcher_info = api.lookup_player(sPitcher_name)
sPitcher_id = sPitcher_info[0]['id']
sPitcher_info_2 = api.get('people',{'personIds':sPitcher_id, 'season':year,'sportId':1})
sPitcher_hand = sPitcher_info_2['people'][0]['pitchHand']['code']

venue_id = game_info[0]['venue_id']

if game_info[0]['status'] == 'Final':
    game_status = True
else:
    game_status = False

print(game_status)
print(game_pk)
print(opp_id)
print(sPitcher_name)
print(sPitcher_id)
print(sPitcher_hand)
print(venue_id)
print(home)

True
745497
134
Jared Jones
683003
R
31
False


fetch player game stats: temp, wind_speed, hp_ump_id, game_time. Fetch list of proj_ump_ids if Home Plate Umpire not yet specified :(

What should happen when this data isnt uploaded yet?

In [909]:
game_boxscore_info = api.boxscore_data(gamePk=game_pk)

temp = None
wind_speed = None
game_time = None

for x in game_boxscore_info['gameBoxInfo']:
    if x.get('label') == 'Weather':
        temp_info = x['value']
        temps = re.findall(r'\d+', temp_info)
        if temps:
            temp = int(temps[0])
            

    if x.get('label') == 'Wind':
        wind_speed_info = x['value']
        winds = re.findall(r'\d+', wind_speed_info)
        if winds:
            wind_speed = int(winds[0])
            

    if x.get('label') == 'First pitch':
        game_time = x['value']
        

    if x.get('label') == 'Umpires':
        ump_info = x['value']
        if 'HP:' in ump_info:
            pattern = r'HP: (.*?)\.'
            match = re.search(pattern, ump_info)
            if match:
                hp_ump = match.group(1)
                hp_ump_info = api.get('jobs_umpires', {})
                for person in hp_ump_info['roster']:
                    if person['person'].get('fullName') == hp_ump:
                        hp_ump_id = person['person'].get('id')
                        print(hp_ump_id)

        else:
            umps = re.findall(r'[\w\s]+', ump_info)
            umps = [ump.strip() for ump in umps if ump.strip]
            ump_list = list(umps) + [None]*(4-len(umps))
            umps_tuple = tuple(ump_list[:4])
            proj_ump_ids = []
            for ump in umps_tuple:
                ump_info = api.get('jobs_umpires',{})
                for person in ump_info['roster']:
                    if person['person'].get('fullName') == ump:
                        ump_id = person['person'].get('id')
                        proj_ump_ids.append(ump_id)
            print(proj_ump_ids)

print(temp)
print(wind_speed)
print(game_time)



605672
80
7
6:42 PM.


fetch player game data: :P
- batting position

In [910]:
game_content = api.boxscore_data(gamePk=game_pk)
if home == True:
    batters_info = game_content['homeBatters']
else:
    batters_info = game_content['awayBatters']

batting_position = None

for batter in batters_info:
    if batter.get('personId') == player_id:
        batting_position = int(batter.get('battingOrder')[0])

print (batting_position)

2


fetch game player stats :P
- If game_status == True
    - gather player stats
- If game_status == False
    - set player stats to None

In [911]:
if game_status == True:
    game_stats_info = api.get('person_stats',{'personId':player_id,'gamePk':game_pk})
    game_stats = game_stats_info['stats'][0]['splits'][2]['stat']
    ab = game_stats.get('atBats')
    h = game_stats.get('hits')
    r = game_stats.get('runs')
    rbi = game_stats.get('rbi')
    hr = game_stats.get('homeRuns')
    tb = game_stats.get('totalBases')
    k = game_stats.get('strikeOuts')
    bb = game_stats.get('baseOnBalls')

else:
    ab = None
    h = None
    r = None
    rbi = None
    hr = None
    tb = None
    k = None
    bb = None


print(f'ab: {ab}')
print(f'h: {h}')
print(f'r: {r}')
print(f'rbi: {rbi}')
print(f'hr: {hr}')
print(f'tb: {tb}')
print(f'k: {k}')
print(f'bb: {bb}')


ab: 4
h: 1
r: 0
rbi: 0
hr: 0
tb: 1
k: 2
bb: 0


Determine Whether Most Recent Streak Averages, or Historical Streak Averages are Needed for Player

Format Player URL for baseballreference.com

In [912]:
initial = last_name[:1]

full_name = f'{first_name} {last_name}'

url = f'https://www.baseball-reference.com/players/{initial}/'
headers = {'User_Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

initial_section = soup.find('div', class_="section_content", id="div_players_")

if initial_section:
    active_players = initial_section.find_all('b')
    for players in active_players:
        player = players.find('a')
        years_playing = players.get_text()[(len(players.get_text()) - 11):]
        if full_name == player.get_text().lower():
            start_year = int(years_playing[1:5])
            end_year = year
            if year in range(start_year, end_year + 1):
                player_link = player['href']
                formatted_player_key = player_link[11:20]

print(formatted_player_key)

ohtansh01


Collect Data for Games Played Up to Desired Date
    - For entirely current year, or entirely last year

In [913]:
season_start_date = api.get('season', {'seasonId': year, 'sportId': 1})['seasons'][0]['regularSeasonStartDate']

if date == season_start_date:
    date = api.get('season', {'seasonId': last_year, 'sportId': 1})['seasons'][0]['postSeasonEndDate']
    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={last_year}#batting_gamelogs'
else:
    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={year}#batting_gamelogs'

day = int(date[8:10])
month = int(date[5:7])

date_dt = d(year, month, day)

new_date_dt = date_dt - timedelta(days=1)

date = new_date_dt.strftime("%Y-%m-%d")

day = int(date[8:10])
month = int(date[5:7])

response = requests.get(search_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('batting_gamelogs'))

if table_rows:

    data = []

    for row in table_rows:

        row_data = {}

        th = row.find('th')
        
        row_num = th.text
        
        row_data['game_num'] = row_num

        for td in row.find_all('td'):

            stat_type = td['data-stat']

            if stat_type == 'date_game':
                stat_value = td['csk'][:10]
            else:
                stat_value = td.text

            row_data[stat_type] = stat_value

        if row_data:

            game_day = int(row_data['date_game'][8:10])
            game_month = int(row_data['date_game'][5:7])

            if game_month < month:
                    data.append(row_data)
            elif game_month == month:
                if game_day <= day:
                     data.append(row_data)

    df = pd.DataFrame(data)

    df.rename(columns={
        'batting_avg': 'BA',
        'onbase_perc': 'OBP',
        'slugging_perc': 'SLG',
    }, inplace=True)

    all_columns = df.columns.tolist()

    columns_to_keep = ['game_num', 'date_game', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP' ]

    columns_to_drop = [column for column in all_columns if column not in columns_to_keep]

    correct_columns_df = df.drop(columns_to_drop, axis=1)

    columns_to_convert_int = ['game_num', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'HBP']

    columns_to_convert_float = ['BA', 'OBP', 'SLG']

    for column in columns_to_convert_int:
         correct_columns_df[column] = correct_columns_df[column].astype(int)

    for column in columns_to_convert_float:
        correct_columns_df[column] = correct_columns_df[column].astype(float)

    correct_columns_df['TB'] = correct_columns_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

    column_order = ['game_num', 'date_game', 'AB', 'H', 'R', 'RBI', '2B', '3B', 'HR', 'TB', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']
    correct_columns_df = correct_columns_df[column_order]

print(correct_columns_df)


    game_num   date_game  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA    OBP  \
0          1  2024-03-20   5  2  0    1   0   0   0   2   0   0  0.400  0.400   
1          2  2024-03-21   5  1  1    1   0   0   0   1   0   0  0.300  0.273   
2          3  2024-03-28   3  2  1    0   1   0   0   3   1   1  0.385  0.400   
3          4  2024-03-29   4  0  0    0   0   0   0   0   0   1  0.294  0.316   
4          5  2024-03-30   5  1  1    0   0   0   0   1   1   2  0.273  0.320   
5          6  2024-03-31   4  1  1    0   1   0   0   2   0   2  0.269  0.310   
6          7  2024-04-01   4  1  1    1   1   0   0   2   0   1  0.267  0.303   
7          8  2024-04-02   3  0  0    0   0   0   0   0   1   1  0.242  0.297   
8          9  2024-04-03   4  2  2    1   0   0   1   5   0   1  0.270  0.317   
9         10  2024-04-05   5  2  2    2   1   0   1   6   0   0  0.286  0.326   
10        11  2024-04-06   4  2  1    0   0   0   0   2   1   1  0.304  0.353   
11        12  2024-04-07   4

Season Averages Up to Desired Date

In [914]:
season_stats = {}

game_data = correct_columns_df.iloc[-1]

games_played = game_data['game_num']

print(games_played)

season_stats['ab_per_game'] = correct_columns_df['AB'].mean()
season_stats['h_per_game'] = correct_columns_df['H'].mean()
season_stats['r_per_game'] = correct_columns_df['R'].mean()
season_stats['rbi_per_game'] = correct_columns_df['RBI'].mean()
season_stats['hr_per_game'] = correct_columns_df['HR'].mean()
season_stats['tb_per_game'] = correct_columns_df['TB'].mean()
season_stats['bb_per_game'] = correct_columns_df['BB'].mean()
season_stats['k_per_game'] = correct_columns_df['SO'].mean()
season_stats['ba_szn'] = game_data['BA']
season_stats['obp_szn'] = game_data['OBP']
season_stats['slg_szn'] = game_data['SLG']

season_stats_df = pd.DataFrame([season_stats])

print(season_stats_df)

58
   ab_per_game  h_per_game  r_per_game  rbi_per_game  hr_per_game  \
0     3.965517    1.275862    0.706897      0.655172     0.241379   

   tb_per_game  bb_per_game  k_per_game  ba_szn  obp_szn  slg_szn  
0     2.362069     0.465517    0.913793   0.322    0.392    0.596  


Streak Average Data Collection

In [915]:
if games_played < 30:
    last_season_end_date = api.get('season', {'seasonId': last_year, 'sportId': 1})['seasons'][0]['postSeasonEndDate']

    day = int(last_season_end_date[8:10])
    month = int(last_season_end_date[5:7])

    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={last_year}#batting_gamelogs'

    response = requests.get(search_url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('batting_gamelogs'))

    if table_rows:

        data = []

        for row in table_rows:

            row_data = {}

            th = row.find('th')
            
            row_num = th.text
            
            row_data['game_num'] = row_num

            for td in row.find_all('td'):

                stat_type = td['data-stat']

                if stat_type == 'date_game':
                    stat_value = td['csk'][:10]
                else:
                    stat_value = td.text

                row_data[stat_type] = stat_value

            if row_data:

                game_day = int(row_data['date_game'][8:10])
                game_month = int(row_data['date_game'][5:7])

                if game_month < month:
                        data.append(row_data)
                elif game_month == month:
                    if game_day <= day:
                        data.append(row_data)

        df = pd.DataFrame(data)

        df.rename(columns={
            'batting_avg': 'BA',
            'onbase_perc': 'OBP',
            'slugging_perc': 'SLG',
        }, inplace=True)

        all_columns = df.columns.tolist()

        columns_to_keep = ['game_num', 'date_game', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']

        columns_to_drop = [column for column in all_columns if column not in columns_to_keep]

        last_season_df = df.drop(columns_to_drop, axis=1)

        columns_to_convert_int = ['game_num', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'HBP']

        columns_to_convert_float = ['BA', 'OBP', 'SLG']

        for column in columns_to_convert_int:
            last_season_df[column] = last_season_df[column].astype(int)

        for column in columns_to_convert_float:
            last_season_df[column] = last_season_df[column].astype(float)

        last_season_df['TB'] = last_season_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

        column_order = ['game_num', 'date_game', 'AB', 'H', 'R', 'RBI', '2B', '3B', 'HR', 'TB', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']
        last_season_df = last_season_df[column_order]

    print(last_season_df)




Last 7 Averages:

In [916]:
if games_played < 7:
    carry_over_games = 7 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_7_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_7_game_num = games_played - 7
    last_7_df = correct_columns_df.iloc[last_7_game_num:]

print(last_7_df)

last_7_stats = {}

last_7_stats['ab_last_7'] = last_7_df['AB'].mean()
last_7_stats['h_last_7'] = last_7_df['H'].mean()
last_7_stats['r_last_7'] = last_7_df['R'].mean()
last_7_stats['rbi_last_7'] = last_7_df['RBI'].mean()
last_7_stats['hr_last_7'] = last_7_df['HR'].mean()
last_7_stats['tb_last_7'] = last_7_df['TB'].mean()
last_7_stats['bb_last_7'] = last_7_df['BB'].mean()
last_7_stats['k_last_7'] = last_7_df['SO'].mean()
last_7_stats['ba_last_7'] = round(last_7_df['H'].sum() / last_7_df['AB'].sum(), 3)
last_7_stats['obp_last_7'] = round((last_7_df['H'].sum() + last_7_df['BB'].sum() + last_7_df['HBP'].sum()) / last_7_df['AB'].sum(), 3)
last_7_stats['slg_last_7'] = round( last_7_df['TB'].sum() / last_7_df['AB'].sum(), 3)

last_7_avg_df = pd.DataFrame([last_7_stats])

print(last_7_avg_df)

    game_num   date_game  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA    OBP  \
51        52  2024-05-25   4  1  0    0   0   1   0   3   0   3  0.338  0.406   
52        53  2024-05-26   4  1  1    0   0   0   0   1   0   1  0.336  0.403   
53        54  2024-05-28   5  0  0    0   0   0   0   0   0   2  0.329  0.395   
54        55  2024-05-29   5  2  1    3   0   0   1   5   0   1  0.330  0.395   
55        56  2024-05-31   3  0  0    0   0   0   0   0   1   1  0.326  0.393   
56        57  2024-06-01   3  1  0    0   0   0   0   1   1   1  0.326  0.395   
57        58  2024-06-02   3  0  0    0   0   0   0   0   1   2  0.322  0.392   

      SLG  HBP  
51  0.628    0  
52  0.621    0  
53  0.606    0  
54  0.615    0  
55  0.607    0  
56  0.604    0  
57  0.596    0  
   ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  bb_last_7  \
0   3.857143  0.714286  0.285714    0.428571   0.142857   1.428571   0.428571   

   k_last_7  ba_last_7  obp_last_7  slg_last_7  
0  

Last 15 Averages

In [917]:
if games_played < 15:
    carry_over_games = 15 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_15_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_15_game_num = games_played - 15
    last_15_df = correct_columns_df.iloc[last_15_game_num:]

print(last_15_df)

last_15_stats = {}

last_15_stats['ab_last_15'] = last_15_df['AB'].mean()
last_15_stats['h_last_15'] = last_15_df['H'].mean()
last_15_stats['r_last_15'] = last_15_df['R'].mean()
last_15_stats['rbi_last_15'] = last_15_df['RBI'].mean()
last_15_stats['hr_last_15'] = last_15_df['HR'].mean()
last_15_stats['tb_last_15'] = last_15_df['TB'].mean()
last_15_stats['bb_last_15'] = last_15_df['BB'].mean()
last_15_stats['k_last_15'] = last_15_df['SO'].mean()
last_15_stats['ba_last_15'] = round(last_15_df['H'].sum() / last_15_df['AB'].sum(), 3)
last_15_stats['obp_last_15'] = round((last_15_df['H'].sum() + last_15_df['BB'].sum() + last_15_df['HBP'].sum()) / last_15_df['AB'].sum(), 3)
last_15_stats['slg_last_15'] = round( last_15_df['TB'].sum() / last_15_df['AB'].sum(), 3)

last_15_avg_df = pd.DataFrame([last_15_stats])

print(last_15_avg_df)

    game_num   date_game  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA    OBP  \
43        44  2024-05-16   2  0  0    0   0   0   0   0   1   1  0.360  0.430   
44        45  2024-05-17   4  1  2    2   0   0   1   4   0   1  0.358  0.426   
45        46  2024-05-18   4  0  0    0   0   0   0   0   0   0  0.350  0.418   
46        47  2024-05-19   4  2  0    1   0   0   0   2   0   1  0.353  0.423   
47        48  2024-05-20   3  0  1    0   0   0   0   0   1   0  0.347  0.419   
48        49  2024-05-21   4  2  2    1   1   0   0   3   0   1  0.351  0.421   
49        50  2024-05-22   4  1  0    0   0   0   0   1   0   0  0.348  0.418   
50        51  2024-05-24   5  0  0    1   0   0   0   0   0   0  0.340  0.409   
51        52  2024-05-25   4  1  0    0   0   1   0   3   0   3  0.338  0.406   
52        53  2024-05-26   4  1  1    0   0   0   0   1   0   1  0.336  0.403   
53        54  2024-05-28   5  0  0    0   0   0   0   0   0   2  0.329  0.395   
54        55  2024-05-29   5

Last 30 Averages

In [918]:
if games_played < 30:
    carry_over_games = 30 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_30_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_30_game_num = games_played - 30
    last_30_df = correct_columns_df.iloc[last_30_game_num:]

print(last_30_df)

last_30_stats = {}

last_30_stats['ab_last_30'] = last_30_df['AB'].mean()
last_30_stats['h_last_30'] = last_30_df['H'].mean()
last_30_stats['r_last_30'] = last_30_df['R'].mean()
last_30_stats['rbi_last_30'] = last_30_df['RBI'].mean()
last_30_stats['hr_last_30'] = last_30_df['HR'].mean()
last_30_stats['tb_last_30'] = last_30_df['TB'].mean()
last_30_stats['bb_last_30'] = last_30_df['BB'].mean()
last_30_stats['k_last_30'] = last_30_df['SO'].mean()
last_30_stats['ba_last_30'] = round(last_30_df['H'].sum() / last_30_df['AB'].sum(), 3)
last_30_stats['obp_last_30'] = round((last_30_df['H'].sum() + last_30_df['BB'].sum() + last_30_df['HBP'].sum()) / last_30_df['AB'].sum(), 3)
last_30_stats['slg_last_30'] = round( last_30_df['TB'].sum() / last_30_df['AB'].sum(), 3)

last_30_avg_df = pd.DataFrame([last_30_stats])

print(last_30_avg_df)

    game_num   date_game  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA    OBP  \
28        29  2024-04-27   5  1  0    1   0   0   0   1   0   1  0.347  0.410   
29        30  2024-04-28   4  0  0    0   0   0   0   0   0   1  0.336  0.399   
30        31  2024-04-29   4  2  1    1   0   0   0   2   1   0  0.341  0.406   
31        32  2024-04-30   5  1  0    0   0   0   0   1   0   3  0.336  0.399   
32        33  2024-05-03   3  1  1    1   0   0   0   1   2   1  0.336  0.405   
33        34  2024-05-04   5  3  2    2   0   0   1   6   0   1  0.345  0.411   
34        35  2024-05-05   4  4  2    3   0   0   2  10   0   0  0.364  0.426   
35        36  2024-05-06   3  2  1    2   0   0   1   5   1   1  0.370  0.434   
36        37  2024-05-07   2  0  1    0   0   0   0   0   2   0  0.365  0.435   
37        38  2024-05-08   4  0  0    0   0   0   0   0   0   2  0.355  0.425   
38        39  2024-05-10   4  2  0    0   1   0   0   3   0   1  0.359  0.427   
39        40  2024-05-11   3

Merge Streak Data

In [919]:
streak_data_df = pd.concat([last_7_avg_df, last_15_avg_df, last_30_avg_df], axis=1)
print(streak_data_df)

   ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  bb_last_7  \
0   3.857143  0.714286  0.285714    0.428571   0.142857   1.428571   0.428571   

   k_last_7  ba_last_7  obp_last_7  slg_last_7  ab_last_15  h_last_15  \
0  1.571429      0.185       0.296        0.37         3.8   0.733333   

   r_last_15  rbi_last_15  hr_last_15  tb_last_15  bb_last_15  k_last_15  \
0   0.466667     0.533333    0.133333    1.333333    0.333333        1.0   

   ba_last_15  obp_last_15  slg_last_15  ab_last_30  h_last_30  r_last_30  \
0       0.193        0.298        0.351         3.9   1.133333   0.566667   

   rbi_last_30  hr_last_30  tb_last_30  bb_last_30  k_last_30  ba_last_30  \
0          0.7    0.233333         2.0    0.433333        1.0       0.291   

   obp_last_30  slg_last_30  
0         0.41        0.513  


Merge Season Averages and Streak Data



In [920]:
batter_season_stats_df = pd.concat([streak_data_df, season_stats_df], axis=1)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(batter_season_stats_df)

   ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  bb_last_7  \
0   3.857143  0.714286  0.285714    0.428571   0.142857   1.428571   0.428571   

   k_last_7  ba_last_7  obp_last_7  slg_last_7  ab_last_15  h_last_15  \
0  1.571429      0.185       0.296        0.37         3.8   0.733333   

   r_last_15  rbi_last_15  hr_last_15  tb_last_15  bb_last_15  k_last_15  \
0   0.466667     0.533333    0.133333    1.333333    0.333333        1.0   

   ba_last_15  obp_last_15  slg_last_15  ab_last_30  h_last_30  r_last_30  \
0       0.193        0.298        0.351         3.9   1.133333   0.566667   

   rbi_last_30  hr_last_30  tb_last_30  bb_last_30  k_last_30  ba_last_30  \
0          0.7    0.233333         2.0    0.433333        1.0       0.291   

   obp_last_30  slg_last_30  ab_per_game  h_per_game  r_per_game  \
0         0.41        0.513     3.965517    1.275862    0.706897   

   rbi_per_game  hr_per_game  tb_per_game  bb_per_game  k_per_game  ba_szn  \
0      0.655

Fetch Starting Pitcher Season Data :(

DEAL WITH DIVIDE BY 0

In [921]:
season_info = api.player_stat_data(personId=sPitcher_id, group='[pitching]', type='yearByYear', sportId=1)

for season in season_info['stats']:
    if int(season['season']) == year:
        season_data = season['stats']

games_started = season_data['gamesStarted']

sp_data = {}

sp_data['sp_games_started'] = games_started
sp_data['sp_pc_per_start'] = round((season_data['numberOfPitches'] / games_started), 3)
sp_data['sp_ip_per_start'] = round((float(season_data['inningsPitched']) / games_started), 3)
sp_data['sp_h_per_start'] = round((season_data['hits'] / games_started), 3)
sp_data['sp_r_per_start'] = round((season_data['runs'] / games_started), 3)
sp_data['sp_hr_per_start'] = round((season_data['homeRuns'] / games_started), 3)
sp_data['sp_bb_per_start'] = round((season_data['baseOnBalls'] / games_started), 3)
sp_data['sp_outs_per_start'] = round((season_data['outs'] / games_started), 3)
sp_data['sp_k_per_start'] = round((season_data['strikeOuts'] / games_started), 3)
sp_data['sp_er_per_start'] = round((season_data['earnedRuns'] / games_started), 3)
sp_data['sp_k_p'] = season_data['strikePercentage']
sp_data['sp_era'] = season_data['era']
sp_data['sp_whip'] = season_data['whip']
sp_data['sp_baa'] = season_data['avg']
sp_data['sp_obpa'] = season_data['obp']
sp_data['sp_slga'] = season_data['slg']
sp_data['sp_pc_per_batter'] = round((season_data['numberOfPitches']) / season_data['battersFaced'], 3)

sp_data_df = pd.DataFrame([sp_data])

print(sp_data_df)


   sp_games_started  sp_pc_per_start  sp_ip_per_start  sp_h_per_start  \
0                12           86.417            5.758           4.583   

   sp_r_per_start  sp_hr_per_start  sp_bb_per_start  sp_outs_per_start  \
0           2.333            0.917             1.25             17.333   

   sp_k_per_start  sp_er_per_start sp_k_p sp_era sp_whip sp_baa sp_obpa  \
0           6.333            2.083   .690   3.25    1.01   .214    .261   

  sp_slga  sp_pc_per_batter  
0    .409             3.757  


Format Batter vs Starting Pitcher URL :P

In [922]:
formatted_batter_name = player_name.replace(' ', '%20')

bvp_url = f'https://baseballsavant.mlb.com/illustrator?playerId={player_id}&playerType=batter&name={formatted_batter_name}&tabSelection=0&shape=square&shapeName=Square%20(Instagram)&subTitle=&chartType=zoneChart-totalPitches&chartName=Zone%20-%20Total%20Pitches&pitcherThrows=&batterStands=&plateAppearanceResults=[]&pitchTypes=[]&seasonType=[%22R%22]&pitchResults=[]&facingPlayer=[%22{sPitcher_id}%22]&balls=[]&battedBallType=[]&years=[{year}]&strikes=[]&outs=[]&selectedGames=[]&homeTeam=[]&awayTeam=[]&vsTeam=[]&exitVelocityGT=0&exitVelocityLT=125&pitchVelocityGT=0&pitchVelocityLT=105&launchAngleGT=-90&launchAngleLT=90&perspective=catcher&venue=7'

Fetch Batter vs Starting Pitcher Stats :(

DEAL WITH DIVIDE BY 0
WHAT IF DATA DOESN"T EXIST YET???

In [923]:
bvp_raw_data = {}
bvp_data = {}

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

driver.get(bvp_url)

driver.implicitly_wait(10)

stats_boxes = driver.find_elements(By.CLASS_NAME, 'stats-box')

for stats_box in stats_boxes:
    stat_type = stats_box.find_element(By.CLASS_NAME, 'stats-title')
    stat_value = stats_box.find_element(By.CLASS_NAME, 'stats-value')
    bvp_raw_data[stat_type.text] = stat_value.text


driver.quit()

bvp_data['vs_sp_pa'] = int(bvp_raw_data['PA'])
bvp_data['vs_sp_p_per_pa'] = round((int(bvp_raw_data['Pitches']) / bvp_data['vs_sp_pa']), 3)
bvp_data['vs_sp_h_per_pa'] = round((int(bvp_raw_data['H']) / bvp_data['vs_sp_pa']), 3)
bvp_data['vs_sp_hr_per_pa'] = round((int(bvp_raw_data['HR']) / bvp_data['vs_sp_pa']), 3)
bvp_data['vs_sp_k_per_pa'] = round((int(bvp_raw_data['SO']) / bvp_data['vs_sp_pa']), 3)
bvp_data['vs_sp_ba_per_pa'] = float(bvp_raw_data['AVG'])
bvp_data['vs_sp_k_p'] = round((float(bvp_raw_data['K%']) / 100), 3)

bvp_data_df = pd.DataFrame([bvp_data])

print(bvp_data_df)



   vs_sp_pa  vs_sp_p_per_pa  vs_sp_h_per_pa  vs_sp_hr_per_pa  vs_sp_k_per_pa  \
0         3           5.333             0.0              0.0           0.667   

   vs_sp_ba_per_pa  vs_sp_k_p  
0              0.0      0.667  


Merge Pitcher data and BvP data :(

should fix itself at some point

In [924]:
sPitcher_stats_df = pd.concat([sp_data_df, bvp_data_df], axis=1)

print(sPitcher_stats_df)

   sp_games_started  sp_pc_per_start  sp_ip_per_start  sp_h_per_start  \
0                12           86.417            5.758           4.583   

   sp_r_per_start  sp_hr_per_start  sp_bb_per_start  sp_outs_per_start  \
0           2.333            0.917             1.25             17.333   

   sp_k_per_start  sp_er_per_start sp_k_p sp_era sp_whip sp_baa sp_obpa  \
0           6.333            2.083   .690   3.25    1.01   .214    .261   

  sp_slga  sp_pc_per_batter  vs_sp_pa  vs_sp_p_per_pa  vs_sp_h_per_pa  \
0    .409             3.757         3           5.333             0.0   

   vs_sp_hr_per_pa  vs_sp_k_per_pa  vs_sp_ba_per_pa  vs_sp_k_p  
0              0.0           0.667              0.0      0.667  


Format URL for Batter vs Opp Data
- If batter has not yet playyed Opp, omit this section from the stats.
- Otherwise, provide per game data for Opp.

Fetch Batter vs Opp Stats

Fetch Umpire Stats
- If Home Plate Ump announced
    - Return data for only HP ump 
- If Home Plate Ump not yet announced
    - Fetch data for all 4 umps
    - hp_ump stats = avg of 4 umps stats

Fetch Venue Stats *WIP